# Analyzing the data

In [168]:
import pandas as pd
import numpy as np

# data 5 is bad and 9 is largest

df = pd.read_csv("odds datas/test_14_odds_data.csv") # odds_data
df["T_goals"] = df["H_score"] + df["A_score"]
df

,week,HT,HT.1,home,draw,away,over_1,under_1,over_2,under_2,over_3,under_3,over_4,under_4,H_score,A_score,T_goals
0,2,WOL,SHU,1.76,3.44,5.15,1.33,3.29,2.08,1.75,3.88,1.26,9.29,1.06,1,0,1
1,2,LUT,MNC,8.62,4.90,1.36,1.21,4.45,1.67,2.20,2.85,1.42,5.92,1.13,0,3,3
2,2,BUR,FUL,2.43,3.24,3.01,1.33,3.31,2.07,1.76,3.85,1.26,9.22,1.06,0,1,1
3,2,FOR,BRN,2.88,3.14,2.59,1.29,3.62,1.93,1.87,3.42,1.31,7.99,1.08,1,3,4
4,2,WHU,BOU,1.77,3.43,5.15,1.27,3.78,1.89,1.91,3.29,1.33,7.54,1.08,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,33,LUT,MNC,8.62,4.90,1.36,1.21,4.45,1.67,2.20,2.85,1.42,5.92,1.13,2,1,3
1446,33,BOU,NWC,3.54,3.00,2.30,1.24,4.05,1.83,1.97,3.07,1.37,7.14,1.09,3,0,3
1447,33,ARS,ASV,1.57,4.03,6.02,1.20,4.56,1.69,2.17,2.84,1.43,6.09,1.12,0,0,0
1448,33,FOR,WHU,2.88,3.14,2.59,1.29,3.62,1.93,1.87,3.42,1.31,7.99,1.08,0,1,1


In [169]:
check_list = []
odd_list = []
for val_1, val_2 in zip(df["over_2"], df["T_goals"]):
    check_list.append((val_1, val_2))
    if len(check_list) == 10:
        # print(check_list)
        sorted_odds = sorted(check_list, key=lambda x: x[0], reverse=True)
        all_odds = [x[0] for x in sorted_odds]
        for odd in sorted_odds:
            if all_odds.count(odd[0]) == 1:
                odd_list.append(odd)
                break               
        check_list = []

print("Total Stake: ", len(odd_list))

# Extract the first indexes into a separate list
first_indexes = [x[0] for x in odd_list]
mean = np.mean(first_indexes)
print("Mean Odd:", np.mean(first_indexes))
# odd_list

Total Stake:  145
Mean Odd: 2.0217931034482755


In [170]:
count = 1
martingale_list = []
for x in odd_list:
    if x[1] > 2:
        # print(count)
        martingale_list.append(count)
        count = 1             
    else:
        count += 1

print(martingale_list)
print(sorted(martingale_list, reverse=True)[:50])
print("Hightest Martingale Level:", max(martingale_list))

[2, 1, 2, 1, 1, 2, 1, 7, 1, 5, 8, 1, 1, 1, 1, 5, 2, 1, 5, 3, 1, 2, 3, 1, 3, 1, 3, 2, 3, 1, 4, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 3, 1, 1, 2, 1, 2, 1, 1, 2, 7, 11, 2, 1, 1, 4, 1, 2, 1, 1, 5, 1, 1, 2]
[11, 8, 7, 7, 5, 5, 5, 5, 4, 4, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Hightest Martingale Level: 11


In [171]:

bankroll = 100000
loss_amount = 3000
highest_loss = -(loss_amount * 3)
print("Highest Loss :", highest_loss)

single_target_prtg = 1
single_target = (single_target_prtg * bankroll) / 100
print("Single Target -", single_target)

total_target_prtg = 3
total_target = (total_target_prtg * bankroll) / 100
print("Total Target -", total_target, "\n")

total_profit = 0
total_loss = 0
for val in martingale_list:
    if val <= 2:
        total_profit += single_target 
        print("WIN :", total_profit)
        if total_profit >= total_target:
            print("---> Target Achieved <---")
    else:
        total_profit -= loss_amount
        print("LOSE :", total_profit)
        if total_profit <= highest_loss:
            print("----------------------------------Stop Loss Reached")

Highest Loss : -9000
Single Target - 1000.0
Total Target - 3000.0 

WIN : 1000.0
WIN : 2000.0
WIN : 3000.0
---> Target Achieved <---
WIN : 4000.0
---> Target Achieved <---
WIN : 5000.0
---> Target Achieved <---
WIN : 6000.0
---> Target Achieved <---
WIN : 7000.0
---> Target Achieved <---
LOSE : 4000.0
WIN : 5000.0
---> Target Achieved <---
LOSE : 2000.0
LOSE : -1000.0
WIN : 0.0
WIN : 1000.0
WIN : 2000.0
WIN : 3000.0
---> Target Achieved <---
LOSE : 0.0
WIN : 1000.0
WIN : 2000.0
LOSE : -1000.0
LOSE : -4000.0
WIN : -3000.0
WIN : -2000.0
LOSE : -5000.0
WIN : -4000.0
LOSE : -7000.0
WIN : -6000.0
LOSE : -9000.0
----------------------------------Stop Loss Reached
WIN : -8000.0
LOSE : -11000.0
----------------------------------Stop Loss Reached
WIN : -10000.0
LOSE : -13000.0
----------------------------------Stop Loss Reached
WIN : -12000.0
WIN : -11000.0
WIN : -10000.0
WIN : -9000.0
WIN : -8000.0
WIN : -7000.0
WIN : -6000.0
WIN : -5000.0
WIN : -4000.0
WIN : -3000.0
WIN : -2000.0
LOSE : -5000

In [173]:
win_length = sorted(martingale_list, reverse=True)
total_diff = 0
for x in win_length:
    if x > 2:
        total_diff += 1
    else:
        break
print("Losses:", total_diff)
print("Wins:", len(martingale_list) - total_diff)

Losses: 16
Wins: 49
